# MNIST - Neural Network
## Xavier initialization

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# nn layers
L1 = torch.nn.Linear(784, 256, bias=True)
L2 = torch.nn.Linear(256, 256, bias=True)
L3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU() 

In [7]:
# Xavier initialization
torch.nn.init.xavier_uniform_(L1.weight)
torch.nn.init.xavier_uniform_(L2.weight)
torch.nn.init.xavier_uniform_(L3.weight)

Parameter containing:
tensor([[-0.0215, -0.0894,  0.0598,  ...,  0.0200,  0.0203,  0.1212],
        [ 0.0078,  0.1378,  0.0920,  ...,  0.0975,  0.1458, -0.0302],
        [ 0.1270, -0.1296,  0.1049,  ...,  0.0124,  0.1173, -0.0901],
        ...,
        [ 0.0661, -0.1025,  0.1437,  ...,  0.0784,  0.0977, -0.0396],
        [ 0.0430, -0.1274, -0.0134,  ..., -0.0582,  0.1201,  0.1479],
        [-0.1433,  0.0200, -0.0568,  ...,  0.0787,  0.0428, -0.0036]],
       requires_grad=True)

In [8]:
# model
model = torch.nn.Sequential(L1, relu, L2, relu, L3).to(device) 

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [10]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch: {:2d}, Cost = {:.6f}'.format(epoch + 1, avg_cost))

print('Learning finished')

Epoch:  1, Cost = 0.250022
Epoch:  2, Cost = 0.094106
Epoch:  3, Cost = 0.061153
Epoch:  4, Cost = 0.043882
Epoch:  5, Cost = 0.033565
Epoch:  6, Cost = 0.025610
Epoch:  7, Cost = 0.020814
Epoch:  8, Cost = 0.017417
Epoch:  9, Cost = 0.018969
Epoch: 10, Cost = 0.013566
Epoch: 11, Cost = 0.011989
Epoch: 12, Cost = 0.013685
Epoch: 13, Cost = 0.009324
Epoch: 14, Cost = 0.011060
Epoch: 15, Cost = 0.009072
Learning finished


In [11]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.targets[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.979699969291687
Label:  1
Prediction:  1
